# Пример работы с allennlp
Здесь мы разберем аналогичную модель, что мы писали на торче.

In [6]:
#!pip uninstall -y allennlp
#!pip install 'allennlp==0.9'

Found existing installation: allennlp 1.2.2
Uninstalling allennlp-1.2.2:
  Successfully uninstalled allennlp-1.2.2
     |████████████████████████████████| 7.6 MB 1.6 MB/s eta 0:00:01
  Using cached flaky-3.7.0-py2.py3-none-any.whl (22 kB)
  Using cached ftfy-5.8.tar.gz (64 kB)


     |████████████████████████████████| 45 kB 201 kB/s eta 0:00:011
     |████████████████████████████████| 158 kB 4.9 MB/s eta 0:00:01
  Using cached responses-0.12.1-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 30.7 MB 93 kB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


  Using cached blis-0.2.4-cp38-cp38-linux_x86_64.whl
  Using cached plac-0.9.6-py2.py3-none-any.whl (20 kB)
  Using cached preshed-2.0.1-cp38-cp38-linux_x86_64.whl
     |████████████████████████████████| 42 kB 48 kB/s  eta 0:00:011
  Using cached thinc-7.0.8-cp38-cp38-linux_x86_64.whl
  Using cached Unidecode-1.1.1-py2.py3-none-any.whl (238 kB)
     |████████████████████████████████| 127 kB 2.3 MB/s eta 0:00:01
  Created wheel for editdistance: filename=editdistance-0.5.3-cp38-cp38-linux_x86_64.whl size=250865 sha256=9a333a73c910dc2c1a337b7bca28d3b62113104feea737e529955bb2308e28bc
  Stored in directory: /home/mlepekhin/.cache/pip/wheels/30/3f/31/5290eec2de81af55384e72a98e523cd8d487d06fcf5509f13a
  Created wheel for ftfy: filename=ftfy-5.8-py3-none-any.whl size=45612 sha256=49f288e1b52f84bc6e40415e171d11c8b13435b7f0e1151569c09d333f5947c6
  Stored in directory: /home/mlepekhin/.cache/pip/wheels/3f/a5/65/684a672b6a26cb8ce3934d155c98d0e23b3dce3d2c0fadae19


  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42709 sha256=3ea9de48a60e3a8e17c5f5ff1541fe9f331222499c2beb673d0de2a13ff29372
  Stored in directory: /home/mlepekhin/.cache/pip/wheels/d8/af/19/fb896f509a437aca2dcf62583e84d7fb2cd5b628c1564a609c
  Created wheel for spacy: filename=spacy-2.1.9-cp38-cp38-linux_x86_64.whl size=49692663 sha256=9b4311fda4d5c0946b8584191445364d57d46c8a373bf045bf04523329350b34
  Stored in directory: /home/mlepekhin/.cache/pip/wheels/50/c2/73/989e659e83b30a74acdfcb825e632b1a7dc12f39d58fe37dd6
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5586 sha256=bb1bf1946eab169ae902a5c9aae3f97ba4df2c6d61d5f6127b3a6a5c48c3846c
  Stored in directory: /home/mlepekhin/.cache/pip/wheels/cb/f3/5a/d88198fdeb46781ddd7e7f2653061af83e7adb2a076d8886d6
Successfully built editdistance ftfy parsimonious spacy word2number
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urll

In [7]:
import torch
from src_allennlp.model import SimpleTagger
from src_allennlp.reader import RuPosReader
from allennlp.data.vocabulary import Vocabulary
from allennlp.common import Params
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

Из всех импортов тут только два самописных класса - ридер и, собственно, сама модель.

In [8]:
reader = RuPosReader()
dataset = reader.read('data/train.csv')
vocab = Vocabulary.from_instances(dataset, pretrained_files={'tokens':'data/cc.ru.300.vec'}, only_include_pretrained_words=False)
print(vocab)
print(vocab.get_index_to_token_vocabulary('labels'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'

В ячейке выше мы прочитали датасет и сделали из него словарь

In [ ]:
word_emb = 300
hidden_dim = 300

params = Params({"pretrained_file": 'data/cc.ru.300.vec', "embedding_dim": word_emb, "trainable": False})
embedder = BasicTextFieldEmbedder({"tokens": Embedding.from_params(vocab, params)})
encoder = PytorchSeq2SeqWrapper(torch.nn.GRU(embedder.get_output_dim(), hidden_dim, batch_first=True, bidirectional=True, num_layers=2))
model = SimpleTagger(vocab, embedder, encoder)

Наша модель состоит из:
- эмбеддера, возвращающего векторные представления для каждого токена
- энкодера, который представляет собой всё ту же GRU, как и раньше, только всякими неудобными вещами занимется `PytorchSeq2SeqWrapper`.
- FeedForward, который уже встроен в модель, но его можно и передать отдельно, предварительно поддержав в самой модели в конструкторе

In [ ]:
train_dataset = dataset[:-1000]
dev_dataset = dataset[-1000:]

Распиливаем датасет, слайсы здесь работают нормально

In [ ]:
device = torch.device('cuda')
model.to(device)
optimizer = torch.optim.Adam(model.parameters())
iterator = BucketIterator(batch_size=256, sorting_keys=[("tokens", "num_tokens")], biggest_batch_first=True)
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=dev_dataset,
                  patience=5,
                  num_epochs=20,
                  cuda_device=0,
                  validation_metric="+fscore")

Переносим модель на видеокарту, задаём оптимизатор, итератор с индексированием по словарю и сам тренер. У тренера есть куча удобных параметров, например, сколько эпох учить, через сколько эпох остановиться, если таргетная метрика перестаёт расти на валидации (`patience`) и собственно сама таргетная метрика, по которой будет выбираться лучшая эпоха

In [ ]:
trainer.train()

После обучения модели остаётся только предстказать метки классов и можно заворачивать

In [ ]:
model.eval()
results = []
with torch.no_grad():    
    labels =  model.forward_on_instance(dev_dataset[1])['labels']

Сам по себе инстанс это что-то похожее на словарь, поэтому чтобы добраться до токенов, нужно немного пройти по словарю

In [ ]:
for token, label in zip(dev_dataset[1]['tokens'].tokens,labels):
    print(token, label)

Собственно вот и все. Allennlp прекрасен и очень удобен